# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes

$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i.
$$

In [1]:
a = rand(10^7) # array of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.520235 
 0.623429 
 0.553711 
 0.0846292
 0.254096 
 0.0659632
 0.0196815
 0.615432 
 0.0771023
 0.62729  
 0.432856 
 0.144667 
 0.967866 
 ⋮        
 0.18709  
 0.590085 
 0.0894427
 0.331779 
 0.652672 
 0.157329 
 0.0313165
 0.571027 
 0.689356 
 0.183919 
 0.844305 
 0.171155 

In [2]:
sum(a) # one expects this is 10^7 * .5 , since the mean of each entry is .5

4.999112117050344e6

# Benchmarking a few ways in a few languages

In [3]:
using BenchmarkTools  # Julia package for benchmarking

#  1. The C language: (8.1 msecs)

C is fften considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

You can put C code in a Julia session, compile it, and run it.

In [4]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = "my_c_library"   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [5]:
c_sum(a)

4.999112117050318e6

In [6]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbol

true

We can now benchmark the C code directly from Julia:

In [7]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.406 ms (0.00% GC)
  median time:      9.960 ms (0.00% GC)
  mean time:        10.971 ms (0.00% GC)
  maximum time:     16.218 ms (0.00% GC)
  --------------
  samples:          454
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [8]:
println("C: Fastest time was $(minimum(c_bench.times)/1e6) msecs.")

C: Fastest time was 9.406151 msecs.


# 2. Python's built in `sum` (68 msecs)

In [9]:
# Julia interface to Python:
using PyCall

In [10]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [11]:
pysum(a)

4.999112117050318e6

In [12]:
pysum(a) ≈ sum(a)

true

In [13]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  672 bytes
  allocs estimate:  19
  --------------
  minimum time:     84.789 ms (0.00% GC)
  median time:      102.456 ms (0.00% GC)
  mean time:        101.037 ms (0.00% GC)
  maximum time:     115.158 ms (0.00% GC)
  --------------
  samples:          50
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [14]:
println("Python (built in): fastest time was $(minimum(py_list_bench.times)/1e6) msecs.")

Python (built in): fastest time was 84.789325 msecs.


# 3. Python: `numpy` (3.9 msec)  

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python

If it is not installed, install it from Julia as follows:

In [15]:
# using Conda 
# Conda.add("numpy")

In [16]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  960 bytes
  allocs estimate:  25
  --------------
  minimum time:     5.464 ms (0.00% GC)
  median time:      5.651 ms (0.00% GC)
  mean time:        5.936 ms (0.00% GC)
  maximum time:     9.713 ms (0.00% GC)
  --------------
  samples:          837
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [17]:
numpy_sum(apy_list) # python thing

4.999112117050342e6

In [18]:
numpy_sum(apy_list) ≈ sum(a)

true

# 4. Python, hand written (419 msec!)

In [19]:
# The PyCall package lets us define python functions directly from Julia:

py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

# mysum_py is a reference to the Python mysum function
mysum_py = py"""mysum"""o

PyObject <function mysum at 0x3359468c0>

In [20]:
@benchmark $mysum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  672 bytes
  allocs estimate:  19
  --------------
  minimum time:     551.920 ms (0.00% GC)
  median time:      591.673 ms (0.00% GC)
  mean time:        594.220 ms (0.00% GC)
  maximum time:     625.041 ms (0.00% GC)
  --------------
  samples:          9
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [21]:
mysum_py(apy_list)

4.999112117050318e6

In [22]:
mysum_py(apy_list) ≈ sum(a)

true

# 5. Julia (built-in) (3.7 msec) 

## Written directly in Julia, not in C!

In [23]:
@which sum(a)

sum(a) at reduce.jl:235

In [24]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.370 ms (0.00% GC)
  median time:      5.645 ms (0.00% GC)
  mean time:        5.944 ms (0.00% GC)
  maximum time:     14.857 ms (0.00% GC)
  --------------
  samples:          835
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

# 6. Julia (hand-written) (8.1 msec, same as hand-written C)

In [25]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [26]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.816 ms (0.00% GC)
  median time:      11.046 ms (0.00% GC)
  mean time:        11.553 ms (0.00% GC)
  maximum time:     18.927 ms (0.00% GC)
  --------------
  samples:          432
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%